# 02. Enriquecimento de Dados: chuva

### 1. Bibliotecas e DataFrame

In [1]:
from config import (
    pd, np, os, salvar_parquet,
    resumo_df, checar_nulos, coord,
    anos, caminho_chuva, url
)

import openmeteo_requests
import requests_cache
import requests
from retry_requests import retry
import time

df = pd.read_parquet("../dados/intermediarios/acidentes_tratado.parquet")

### 2. Configura API Open-Meteo com cache e retry em caso de erro 

In [2]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

### 3. Cria função para chamar API

Função para coletar dados horários de precipitação de um ponto (lat/lon)
para um determinado ano, salvando em formato parquet.

Args:  
* lat (float): Latitude
* lon (float): Longitude
* nome (str): Nome da região (ex.: NORTE)


In [3]:
def dados_chuva(lat: float, lon: float, nome: str):
   
    os.makedirs(caminho_chuva, exist_ok=True)

    for ano in anos:
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": f"{ano}-01-01",
            "end_date": f"{ano}-12-31",
            "hourly": 'precipitation',
            "timezone": 'America/Sao_Paulo'
        }

        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]
        hourly = response.Hourly()

        chuva_hora = hourly.Variables(0).ValuesAsNumpy()

        df_chuva = pd.DataFrame({
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ),
            "precipitation": chuva_hora
        })

        nome_arquivo = f"{caminho_chuva}{nome.lower()}_{ano}.parquet"
        salvar_parquet(df_chuva, nome_arquivo)
    return df_chuva

### 4. Chama API por região

#### 1. Norte

In [7]:
lat, lon = coord["NORTE"]
dados_chuva(lat, lon, "NORTE")
df_teste = pd.read_parquet(f"{caminho_chuva}norte_2021.parquet")
resumo_df(df_teste)

Salvo: ../dados/intermediarios/clima/norte_2020.parquet
Salvo: ../dados/intermediarios/clima/norte_2021.parquet
Salvo: ../dados/intermediarios/clima/norte_2022.parquet
Salvo: ../dados/intermediarios/clima/norte_2023.parquet
Salvo: ../dados/intermediarios/clima/norte_2024.parquet
Dimensões: (8760, 2)

Tipos de dados:
date             datetime64[ns, UTC]
precipitation                float32
dtype: object

Nulos por coluna:
date                0
precipitation    1942
dtype: int64


,date,precipitation
0,2021-01-01 03:00:00+00:00,NaN
1,2021-01-01 04:00:00+00:00,NaN
2,2021-01-01 05:00:00+00:00,NaN
3,2021-01-01 06:00:00+00:00,NaN
4,2021-01-01 07:00:00+00:00,NaN


#### 2. Leste

In [24]:
lat, lon = coord["LESTE"]
dados_chuva(lat, lon, "LESTE")

✅ DataFrame salvo em ../dados/intermediarios/clima/leste_2020.parquet
Salvo: ../dados/intermediarios/clima/leste_2020.parquet
✅ DataFrame salvo em ../dados/intermediarios/clima/leste_2021.parquet
Salvo: ../dados/intermediarios/clima/leste_2021.parquet
✅ DataFrame salvo em ../dados/intermediarios/clima/leste_2022.parquet
Salvo: ../dados/intermediarios/clima/leste_2022.parquet
✅ DataFrame salvo em ../dados/intermediarios/clima/leste_2023.parquet
Salvo: ../dados/intermediarios/clima/leste_2023.parquet
✅ DataFrame salvo em ../dados/intermediarios/clima/leste_2024.parquet
Salvo: ../dados/intermediarios/clima/leste_2024.parquet


,date,precipitation
0,2024-01-01 03:00:00+00:00,0.00
1,2024-01-01 04:00:00+00:00,0.00
2,2024-01-01 05:00:00+00:00,0.00
3,2024-01-01 06:00:00+00:00,0.00
4,2024-01-01 07:00:00+00:00,0.00
...,...,...
8779,2024-12-31 22:00:00+00:00,0.00
8780,2024-12-31 23:00:00+00:00,0.00
8781,2025-01-01 00:00:00+00:00,0.00
8782,2025-01-01 01:00:00+00:00,0.00


#### 3. Centro

In [26]:
lat, lon = coord["CENTRO"]
dados_chuva(lat, lon, "CENTRO")

✅ DataFrame salvo em ../dados/intermediarios/clima/centro_2020.parquet
✅ DataFrame salvo em ../dados/intermediarios/clima/centro_2021.parquet
✅ DataFrame salvo em ../dados/intermediarios/clima/centro_2022.parquet
✅ DataFrame salvo em ../dados/intermediarios/clima/centro_2023.parquet
✅ DataFrame salvo em ../dados/intermediarios/clima/centro_2024.parquet


,date,precipitation
0,2024-01-01 03:00:00+00:00,0.00
1,2024-01-01 04:00:00+00:00,0.00
2,2024-01-01 05:00:00+00:00,0.00
3,2024-01-01 06:00:00+00:00,0.00
4,2024-01-01 07:00:00+00:00,0.00
...,...,...
8779,2024-12-31 22:00:00+00:00,0.20
8780,2024-12-31 23:00:00+00:00,0.00
8781,2025-01-01 00:00:00+00:00,0.00
8782,2025-01-01 01:00:00+00:00,0.00


#### 4. Sul

In [18]:
lat, lon = coord["SUL"]
dados_chuva(lat, lon, "SUL")

✅ Arquivo salvo: ../dados/intermediarios/clima/sul_2020.parquet
✅ Arquivo salvo: ../dados/intermediarios/clima/sul_2021.parquet
✅ Arquivo salvo: ../dados/intermediarios/clima/sul_2022.parquet
✅ Arquivo salvo: ../dados/intermediarios/clima/sul_2023.parquet
✅ Arquivo salvo: ../dados/intermediarios/clima/sul_2024.parquet


,date,precipitation
0,2024-01-01 03:00:00+00:00,0.00
1,2024-01-01 04:00:00+00:00,0.00
2,2024-01-01 05:00:00+00:00,0.00
3,2024-01-01 06:00:00+00:00,0.00
4,2024-01-01 07:00:00+00:00,0.00
...,...,...
8779,2024-12-31 22:00:00+00:00,0.10
8780,2024-12-31 23:00:00+00:00,0.00
8781,2025-01-01 00:00:00+00:00,0.00
8782,2025-01-01 01:00:00+00:00,0.00
